# Project 3 – IMDB MOVIE REVIEW

In [1]:
# coding=utf-8
import os
import math
import time
import joblib
import random
import numpy as np
import pandas as pd
import random as rd
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import category_encoders as ce
from time import time

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from xgboost import *
from scipy.stats import normaltest, linregress
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *
from sklearn.cluster import *
from sklearn.ensemble import *
from sklearn.datasets import load_digits, make_hastie_10_2, load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import *
from sklearn.decomposition import *
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import *
from sklearn.feature_selection import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from pyclustering.cluster.kmeans import kmeans
from pyclustering.utils.metric import distance_metric, type_metric
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
import unicodedata
import warnings
import re
from bs4 import BeautifulSoup
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression, mutual_info_classif, SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from nltk import corpus, RegexpTokenizer, word_tokenize, download, ToktokTokenizer
import nltk
import re
import numpy as np
import pandas as pd
from nltk.tokenize.toktok import ToktokTokenizer
from nltk import corpus, RegexpTokenizer, word_tokenize, download, ToktokTokenizer
import spacy.cli

# %matplotlib
# %matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 5000)

nlp = spacy.load('en_core_web_sm')

2022-11-29 14:37:01.538115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class TextPreprocessing:
    def __init__(self):
        download('punkt')
        self.stopword_list = corpus.stopwords.words('english')
        additional_stopwords = ['also', 'fd', 'nan', 'oh', 'ok', 'ms', 'mr', 'aa', 'yet', 'become', 'fd', 'ol', 'em', 'hey',
                               'wb', 'ct']
        self.stopword_list.append(additional_stopwords)
        self.stopword_list.remove('not')

        # see: https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/bonus%20content/nlp%20proven%20approach/contractions.py
        self.CONTRACTION_MAP = {
            "ain't": "is not",
            "aren't": "are not",
            "can't": "cannot",
            "can't've": "cannot have",
            "'cause": "because",
            "could've": "could have",
            "couldn't": "could not",
            "couldn't've": "could not have",
            "didn't": "did not",
            "doesn't": "does not",
            "don't": "do not",
            "hadn't": "had not",
            "hadn't've": "had not have",
            "hasn't": "has not",
            "haven't": "have not",
            "he'd": "he would",
            "he'd've": "he would have",
            "he'll": "he will",
            "he'll've": "he he will have",
            "he's": "he is",
            "how'd": "how did",
            "how'd'y": "how do you",
            "how'll": "how will",
            "how's": "how is",
            "I'd": "I would",
            "I'd've": "I would have",
            "I'll": "I will",
            "I'll've": "I will have",
            "I'm": "I am",
            "I've": "I have",
            "i'd": "i would",
            "i'd've": "i would have",
            "i'll": "i will",
            "i'll've": "i will have",
            "i'm": "i am",
            "i've": "i have",
            "isn't": "is not",
            "it'd": "it would",
            "it'd've": "it would have",
            "it'll": "it will",
            "it'll've": "it will have",
            "it's": "it is",
            "let's": "let us",
            "ma'am": "madam",
            "mayn't": "may not",
            "might've": "might have",
            "mightn't": "might not",
            "mightn't've": "might not have",
            "must've": "must have",
            "mustn't": "must not",
            "mustn't've": "must not have",
            "needn't": "need not",
            "needn't've": "need not have",
            "o'clock": "of the clock",
            "oughtn't": "ought not",
            "oughtn't've": "ought not have",
            "shan't": "shall not",
            "sha'n't": "shall not",
            "shan't've": "shall not have",
            "she'd": "she would",
            "she'd've": "she would have",
            "she'll": "she will",
            "she'll've": "she will have",
            "she's": "she is",
            "should've": "should have",
            "shouldn't": "should not",
            "shouldn't've": "should not have",
            "so've": "so have",
            "so's": "so as",
            "that'd": "that would",
            "that'd've": "that would have",
            "that's": "that is",
            "there'd": "there would",
            "there'd've": "there would have",
            "there's": "there is",
            "they'd": "they would",
            "they'd've": "they would have",
            "they'll": "they will",
            "they'll've": "they will have",
            "they're": "they are",
            "they've": "they have",
            "to've": "to have",
            "wasn't": "was not",
            "we'd": "we would",
            "we'd've": "we would have",
            "we'll": "we will",
            "we'll've": "we will have",
            "we're": "we are",
            "we've": "we have",
            "weren't": "were not",
            "what'll": "what will",
            "what'll've": "what will have",
            "what're": "what are",
            "what's": "what is",
            "what've": "what have",
            "when's": "when is",
            "when've": "when have",
            "where'd": "where did",
            "where's": "where is",
            "where've": "where have",
            "who'll": "who will",
            "who'll've": "who will have",
            "who's": "who is",
            "who've": "who have",
            "why's": "why is",
            "why've": "why have",
            "will've": "will have",
            "won't": "will not",
            "won't've": "will not have",
            "would've": "would have",
            "wouldn't": "would not",
            "wouldn't've": "would not have",
            "y'all": "you all",
            "y'all'd": "you all would",
            "y'all'd've": "you all would have",
            "y'all're": "you all are",
            "y'all've": "you all have",
            "you'd": "you would",
            "you'd've": "you would have",
            "you'll": "you will",
            "you'll've": "you will have",
            "you're": "you are",
            "you've": "you have"
        }

        self.doc = pd.read_excel("IMDB_dataset.xlsx")
        self.test = "It's Eighty-seven NLP jobs in 1950 ~ 1960s, yet. ,  Onward!"

    def expand_match(self, contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = self.CONTRACTION_MAP.get(match) \
            if self.CONTRACTION_MAP.get(match) \
            else self.CONTRACTION_MAP.get(match.lower())
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction

    def process_contraction(self, line):
        contractions_pattern = re.compile('({})'.format('|'.join(self.CONTRACTION_MAP.keys())), flags=re.IGNORECASE | re.DOTALL)
        return contractions_pattern.sub(self.expand_match, line)

    def remove_punctuations(self, line):
        # remove punctuations
        punctuation = '\[\]\.!,;:?"\'\(\)\*-[0-9]\{1-3\}'
        return re.sub(r'[{}]+'.format(punctuation), ' ', line)

    def process_ToktokTokenizer(self, line):
        tokenizer = ToktokTokenizer()
        tokens = tokenizer.tokenize(line)
        tokens = [token.strip() for token in tokens]
        return np.array(tokens)

    def process_ToktokTokenizer(self, line):
        tokenizer = ToktokTokenizer()
        tokens = tokenizer.tokenize(line)
        tokens = [token.strip() for token in tokens]
        return np.array(tokens)

    def remove_stopwords(self, tokens):
        tks = [token for token in tokens if token not in self.stopword_list and len(token)!=1]
        sws = [token for token in tokens if token in self.stopword_list]            
        return str(tks)  # , np.array(sws)
    
    
if __name__ == '__main__':
    foo = TextPreprocessing()

[nltk_data] Downloading package punkt to /Users/francis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
foo.doc.head()

,review,sentiment
0,I thought this was a wonderful way to spend ti...,positive
1,"Probably my all-time favorite movie, a story o...",positive
2,I sure would like to see a resurrection of a u...,positive
3,"This show was an amazing, fresh & innovative i...",negative
4,Encouraged by the positive comments about this...,negative


### There are 25000 rows of text data in this document, the label are balanced

In [4]:
foo.doc.shape

(25000, 2)

In [5]:
foo.doc.groupby('sentiment').count()

,review
sentiment,
negative,12500
positive,12500


### Based on the description, I split the dataset into training set, and test set, equaly

In [6]:
X = foo.doc['review']
y = foo.doc['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0, stratify=y)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_train

0        This was a terrible film. There was no story l...
1        The story line has been rehashed a number of t...
2        LCDR Tom Dodge, despite having a reputation am...
3        I suppose for 1961 this film was supposed to b...
4        I strongly disagree with "ctomvelu" regarding ...
                               ...                        
12495    The Emperor's New Groove was a great twist for...
12496    Ashley Judd, in an early role and I think her ...
12497    As a huge fan or the Cracker series, I have be...
12498    Another Day - this movie requires you to watch...
12499    Maybe I saw a cleaned up version, but other th...
Name: review, Length: 12500, dtype: object

##### Training set are balanced

In [7]:
# y_train label balance
pd.DataFrame(y_train).reset_index().groupby('sentiment').count()

,index
sentiment,
negative,6250
positive,6250


##### Test set are balanced

In [8]:
# y_test label balance
pd.DataFrame(y_test).reset_index().groupby('sentiment').count()

,index
sentiment,
negative,6250
positive,6250


### Applying preprocessing steps on 5000 samples for tuning all the processes that needed

In [9]:
X = foo.doc.head(5000)['review']
y = foo.doc.head(5000)['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0, stratify=y)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

X_train = X_train.apply(lambda l: BeautifulSoup(l, "html.parser").get_text())
X_train = X_train.apply(lambda l: unicodedata.normalize('NFKD', l))
X_train = X_train.apply(lambda l: re.sub(r'[^a-zA-Z0-9\s]', '', l))
X_train = X_train.apply(lambda l: re.sub(r'[0-9]+', '', l))
X_train = X_train.apply(lambda l: re.sub('\[[^]]*\]', '', l))
X_train = X_train.apply(lambda l: l.lower())
# X_train = X_train.apply(lambda l: ' '.join([nltk.porter.PorterStemmer().stem(word) for word in l.split()]))
X_train = X_train.apply(lambda l: ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in nlp(l)]))
X_train = X_train.apply(lambda l: foo.process_contraction(l))
X_train = X_train.apply(lambda l: foo.remove_punctuations(l))
X_train = X_train.apply(lambda l: foo.remove_stopwords(foo.process_ToktokTokenizer(l)))
X_train

0       ['adaptation', 'dirk', 'wittenborn', 'book', '...
1       ['crappy', 'movie', 'idea', 'get', 'shelf', 'm...
2       ['want', 'like', 'movie', 'little', 'like', 's...
3       ['surprising', 'production', 'like', 'gets', '...
4       ['convict', 'serve', 'time', 'come', 'forward'...
                              ...                        
2495    ['one', 'time', 'favorite', 'simple', 'sweet',...
2496    ['star', 'trek', 'motion', 'picture', 'mostly'...
2497    ['movie', 'try', 'capture', 'essence', 'surrea...
2498    ['work', 'video', 'store', 'get', 'see', 'one'...
2499    ['child', 'watch', 'show', 'everyday', 'great'...
Name: review, Length: 2500, dtype: object

In [10]:
# Instantiate and fit a TFIDF vectorizer and then use that trained vectorizer
# to transform the messages in the training and test sets
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train.values)
X_train_vect = tfidf_vect.transform(X_train.values)
X_test_vect = tfidf_vect.transform(X_test.values)

##### Examming the TF-IDF vocabulary to find out extra stopwords and other problems

In [11]:
tfidf_vect.vocabulary_

{'adaptation': 294,
 'dirk': 7070,
 'wittenborn': 29748,
 'book': 2994,
 'not': 18602,
 'read': 21635,
 'young': 30166,
 'finn': 9631,
 'earl': 7847,
 'live': 15620,
 'mom': 17401,
 'liz': 15635,
 'diane': 6898,
 'lane': 15006,
 'cramp': 5744,
 'low': 15873,
 'east': 7875,
 'side': 24164,
 'new': 18297,
 'york': 30151,
 'apartment': 1260,
 'dream': 7598,
 'join': 14264,
 'anthropologist': 1155,
 'father': 9240,
 'study': 25660,
 'fierce': 9448,
 'tribe': 27529,
 'south': 24889,
 'america': 919,
 'boyfriend': 3132,
 'coke': 4926,
 'catch': 4041,
 'score': 23301,
 'one': 18988,
 'customer': 6077,
 'legitimate': 15260,
 'masseuse': 16515,
 'rich': 22355,
 'mr': 17791,
 'osborne': 19220,
 'bail': 1994,
 'return': 22263,
 'full': 10311,
 'time': 27025,
 'personal': 20031,
 'huge': 12735,
 'estate': 8577,
 'jersey': 14160,
 'drive': 7645,
 'limo': 15536,
 'string': 25611,
 'lie': 15437,
 'back': 1916,
 'seat': 23439,
 'dress': 7621,
 'hitch': 12380,
 'way': 29228,
 'pantie': 19616,
 'show': 

In [12]:
# additional_stopwords = ['also', 'fd', 'nan', 'oh', 'ok', 'ms', 'mr', 'aa', 'yet', 'become', 'fd', 'ol', 'em', 'hey',
#                        'wb', 'ct']
# foo.stopword_list.append(additional_stopwords)

##### Applying Random Forest with default hyper-parameters to evaluate the quality of the preprocessing process

Go back to add more steps to make a better evaluating score.

In [13]:
# Fit a basic Random Forest model on these vectors
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect, y_train.values.flatten())
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect)


In [14]:
# Evaluate the predictions of the model on the holdout test set
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test, y_pred, pos_label="positive")
recall = recall_score(y_test, y_pred, pos_label="positive")

print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision: 0.798 / Recall: 0.866 / Accuracy: 0.823


### Model training on normalized corpus

In [19]:
class Driver:
    def __init__(self, testing=False, frac=0.01, test_split_size=0.1):
        self.foo = TextPreprocessing()
        if testing:
            self.foo.doc = self.foo.doc.sample(frac=frac, replace=False, random_state=0)
            print('DOWN SAMPLING:')
        print("sample size: %s" % str(self.foo.doc.shape))
        
        r = random.randint(1, 10000)
        print("test site split random_state : %d" % r)
        self.X = self.foo.doc['review']
        self.y = self.foo.doc['sentiment']
        self.X_train, self.X_test, self.y_train, self.y_test = \
            train_test_split(self.X, self.y, test_size=test_split_size, random_state=r, stratify=self.y)
        print("training data size: %s" % str(self.X_train.shape))
        
        self.X_train = self.X_train.reset_index(drop=True)
        self.X_test = self.X_test.reset_index(drop=True)
        pass

    def text_processing(self):
        print('remove html structures ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: BeautifulSoup(l, "html.parser").get_text())
        print('time cost %.2fs' % (time() - t))
        print('unicodedata to NFKD ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: unicodedata.normalize('NFKD', l))
        print('time cost %.2fs' % (time() - t))
        print('remove single charactors ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: re.sub(r'[^a-zA-Z0-9\s]', '', l))
        print('time cost %.2fs' % (time() - t))
        print('remove all the numbers ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: re.sub(r'[0-9]+', '', l))
        print('time cost %.2fs' % (time() - t))
        print('remove punctuation structures ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: re.sub('\[[^]]*\]', '', l))
        print('time cost %.2fs' % (time() - t))
        print('lower case ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: l.lower())
        print('time cost %.2fs' % (time() - t))
#         print('stemming ... ', end='')
#         t = time()
#         self.X_train = self.X_train.apply(lambda l: ' '.join([nltk.porter.PorterStemmer().stem(word) for word in l.split()]))
#         print('time cost %.2fs' % (time() - t))
        print('lemmatization ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in nlp(l)]))
        print('time cost %.2fs' % (time() - t))
        print('expand contractions ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: self.foo.process_contraction(l))
        print('time cost %.2fs' % (time() - t))
        print('remove all punctuations ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: self.foo.remove_punctuations(l))
        print('time cost %.2fs' % (time() - t))
        print('tokenization ... ', end='')
        t = time()
        self.X_train = self.X_train.apply(lambda l: self.foo.remove_stopwords(self.foo.process_ToktokTokenizer(l)))
        print('time cost %.2fs' % (time() - t))
        print()
        pass

    def tfidf_vectorizer(self):
        tfidf_vect = TfidfVectorizer()
        tfidf_vect.fit(self.X_train.values)
        X_train_vect = tfidf_vect.transform(self.X_train.values)
        X_test_vect = tfidf_vect.transform(self.X_test.values)
        return X_train_vect, X_test_vect

    def model_select(self, results):
        print("Now the best model is")
        best_model_name = None
        best_accuracy = 0.0
        self.__print_title()
        for _, tup in enumerate(results):
            self.__print_result(tup)
            name = tup[0]
            accuracy = tup[1][2]
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model_name = name
        
        algo = best_model_name.split(' ')[0]
        print(
            "%s received the highest average performance with accuracy %.4f. Therefore %s is the most suitable model.\n" % (
            algo, best_accuracy, algo))

    def evaluating(self, callable_, verbose=0):
        self.__print_title()
        buffer = []
        name = None
        # pred_lst = []
        for _ in range(5):
            name, clf = callable_(self.X_train, self.y_train,
                                  random_state=random.randint(1, 10000), verbose=verbose)
            start = time()
            y_pred = clf.predict(self.X_test)
            # pred_lst.append(y_pred)
            cost = (time() - start) * 1000
            # tmp_params = clf.cv_results_['params'][clf.best_index_]
            tup = (
                round(clf.cv_results_['mean_test_score'][clf.best_index_], 3),
                round(clf.cv_results_['std_test_score'][clf.best_index_] * 2, 3),
                # round(accuracy_score(self.y_test, y_pred), 3),
                round((y_pred==self.y_test).sum()/len(y_pred), 3),
                round(precision_score(self.y_test, y_pred, pos_label="positive"), 3),
                round(recall_score(self.y_test, y_pred, pos_label="positive"), 3),
                round(cost, 3),
            )
            buffer.append(np.array(tup))
            self.__print_result((name, tup))
        tup = np.array(buffer).mean(axis=0)
        self.__print_result(("%s avg" % name, tup))
        print()
        # self.df[name] = pd.DataFrame(pred_lst).mode().values.flatten()
        return "%s avg" % name, tup

    @staticmethod
    def __print_title():
        print("%15s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s" % (
            "model", "mean", "std", "accuracy", "precision", "recall", "latency"))

    @staticmethod
    def __print_result(tup):
        name, r = tup
        if name.find("avg") >= 0:
            std = ""
        else:
            std = "+/-%.3f" % r[1]
        print("%15s\t%10s\t%10s\t%10s\t%10s\t%10s\t%10s" % (
            "%s" % name, "%.3f" % r[0], std, "%.3f" % r[2], "%.3f" % r[3], "%.3f" % r[4], "%.3f ms" % r[5]))

    def evaluating_model(self, clf):
        pred = clf.predict(self.X_test)
        print('Training set score: {:.4f}'.format(clf.score(self.X_train, self.y_train)))
        print('Test set score: {:.4f}'.format(clf.score(self.X_test, self.y_test)))
        print('Model accuracy score: {0:0.4f}'.format(accuracy_score(self.y_test, pred)))
        print(classification_report(self.y_test, pred))
        pass
    
    def train_LR(self, x, y, random_state=0, verbose=0):
        model = LogisticRegression()
        params = {
            'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 1e4]
        }
        k_fold = StratifiedKFold(5, shuffle=True, random_state=random_state)
        clf = self.search_cv(model, params, k_fold, verbose=verbose)
        clf.fit(x, y)
        return "LR", clf

    def train_DT(self, x, y, random_state=0, verbose=0):
        model = DecisionTreeClassifier()
        params = {
            'max_depth': [2, 3, 5, 10, 20],
            'min_samples_leaf': [5, 10, 20, 50, 100],
            'criterion': ["gini", "entropy"]
        }
        k_fold = StratifiedKFold(5, shuffle=True, random_state=random_state)
        clf = self.search_cv(model, params, k_fold, verbose=verbose)
        clf.fit(x, y)
        return "DT", clf

    def train_NB(self, x, y, random_state=0, verbose=0):
        model = GaussianNB()
        params = {
            'var_smoothing': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]
        }
        k_fold = StratifiedKFold(5, shuffle=True, random_state=random_state)
        clf = self.search_cv(model, params, k_fold, verbose=verbose)
        clf.fit(x, y)
        return "NB", clf
    
    def train_RF(self, x, y, random_state=0, verbose=0):
        model = RandomForestClassifier()
        params = {
            'bootstrap': [True, False],
            'max_depth': [2, 4, 8, 16, 32],
            'max_features': ['auto', 'sqrt', 'log2'],
            'min_samples_leaf': [1, 2, 4],
            'min_samples_split': [2, 5, 10],
            'n_estimators': [10, 20, 50, 100, 200],
            'criterion': ['gini', 'entropy']
        }
        # scoring = {
        #     "Accuracy": make_scorer(accuracy_score),
        #     "mean_absolute_error": make_scorer(mean_absolute_error),
        #     "mean_squared_error": make_scorer(mean_squared_error),
        #     "r2_score": make_scorer(r2_score),
        # }
        k_fold = StratifiedKFold(5, shuffle=True, random_state=random_state)
        clf = self.search_cv(model, params, k_fold, verbose=verbose)
        clf.fit(x, y)
        return "RF", clf

    def train_XGB(self, x, y, random_state=0, verbose=0):
        model = XGBClassifier(eval_metric='logloss')
        params = {
            "learning_rate": [0.001, 0.01, 0.1, 1],
            "max_depth": [2,4,6,8,10],
            "min_child_weight": range(1, 10),
            "gamma": np.arange(0, 0.7, 0.2),
            "colsample_bytree": np.arange(0.1, 1.1, 0.1),
            "n_estimators": [10,20,50,100,200,500]
        }
        k_fold = StratifiedKFold(5, shuffle=True, random_state=random_state)
        clf = self.search_cv(model, params, k_fold, verbose=verbose)
        clf.fit(x, y)
        return "XGB", clf
        
    @staticmethod
    def search_cv(model, params, fold, verbose=0):
        return RandomizedSearchCV(model, params, cv=fold, verbose=verbose, return_train_score=True, )
        # return RandomizedSearchCV(model, params, cv=fold, verbose=0,
        #                           scoring=scoring, refit=list(scoring.items())[0][0], return_train_score=True, )


go = Driver(testing=False, frac=0.2, test_split_size=0.5)

[nltk_data] Downloading package punkt to /Users/francis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


sample size: (25000, 2)
test site split random_state : 9314
training data size: (12500,)


##### Preprocess text data

In [20]:
go.text_processing()

remove html structures ... time cost 1.71s
unicodedata to NFKD ... time cost 0.04s
remove single charactors ... time cost 0.24s
remove all the numbers ... time cost 0.31s
remove punctuation structures ... time cost 0.01s
lower case ... time cost 0.02s
lemmatization ... time cost 310.60s
expand contractions ... time cost 16.59s
remove all punctuations ... time cost 0.10s
tokenization ... time cost 13.41s



##### Perform TF-IDF Vectorization

In [21]:
go.X_train, go.X_test = go.tfidf_vectorizer()

##### Model Selection with RandomizedSearchCV and Stratifie K-Fold

In [22]:
arr = [
    go.evaluating(go.train_XGB, verbose=0),
    go.evaluating(go.train_RF, verbose=0),
]

          model	      mean	       std	  accuracy	 precision	    recall	   latency
            XGB	     0.834	  +/-0.014	     0.801	     0.751	     0.899	 75.715 ms
            XGB	     0.843	  +/-0.018	     0.817	     0.775	     0.893	 81.959 ms
            XGB	     0.852	  +/-0.008	     0.829	     0.785	     0.904	 85.254 ms
            XGB	     0.837	  +/-0.007	     0.806	     0.757	     0.901	 85.967 ms
            XGB	     0.828	  +/-0.019	     0.810	     0.781	     0.861	 74.601 ms
        XGB avg	     0.839	          	     0.813	     0.770	     0.892	 80.699 ms

          model	      mean	       std	  accuracy	 precision	    recall	   latency
             RF	     0.826	  +/-0.010	     0.787	     0.727	     0.919	696.998 ms
             RF	     0.847	  +/-0.016	     0.805	     0.741	     0.936	888.103 ms
             RF	     0.829	  +/-0.004	     0.783	     0.720	     0.927	363.059 ms
             RF	     0.839	  +/-0.014	     0.802	     0.738	     0.938	715.411 ms
             RF

##### Report of the best performing model

In [23]:
go.model_select(arr)

Now the best model is
          model	      mean	       std	  accuracy	 precision	    recall	   latency
        XGB avg	     0.839	          	     0.813	     0.770	     0.892	 80.699 ms
         RF avg	     0.836	          	     0.794	     0.731	     0.931	686.250 ms
XGB received the highest average performance with accuracy 0.8126. Therefore XGB is the most suitable model.



### Conclusion

A normalized corpus is required for NLP Process. So I applied several steps to raw text to normalize it. Besides, high-frequency stopwords will have a detrimental impact on the model's ability to predict outcomes. To further enhance the document preprocessing, I looked through the TF-IDF vector vocabulary to add some more stopwords to the stopwords list. Then I apply Random Forest on this normalized corpus to assess how well the preparation was done and repeat previous steps to enhance the performance of the corpus.

Applying lemmatization or stemming is very helpful for targeting the intended meaning of context. And after some experiments, I decide to apply lemmatization nor stemming for a better accuracy score, about 2%.

After all the efforts shows in this module, XGB received the highest average performance with accuracy 0.8126. Therefore, XGB is the most suitable model.
